In [20]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary


In [2]:
hidden_size = 128

In [3]:
def load_data(path):
    df = pd.read_csv(path, header=None)
    X = df[0].values
    y = df[1].values
    x_tok = Tokenizer(char_level=True, filters='')
    x_tok.fit_on_texts(X)
    y_tok = Tokenizer(char_level=True, filters='')
    y_tok.fit_on_texts(y)
    
    X = x_tok.texts_to_sequences(X)
    y = y_tok.texts_to_sequences(y)
    
    X = pad_sequences(X)
    y = np.asarray(y)
    
    return X, y, x_tok.word_index, y_tok.word_index

In [4]:
X, y, x_wid, y_wid= load_data('data/data.csv')

In [1]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
    
    def forward(self, input, hidden):
        embedded = self.embedding(input)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden # BxSxH, Bx1xH
    
    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)                

class Attn(nn.Module):
    def __init__(self, hidden_size):
        super(Attn ,self).__init__()
        
    def forward(self, hidden, encoder_outputs):
        # encoder_outputs: BxSxH
        # hidden: Bx1xH
        hidden = torch.tranpose(hidden, 1, 2) # BxHx1
        energies = torch.bmm(encoder_outputs, hidden).squeeze(-1) # BxS
        attn_weights = F.softmax(energies).unsqueeze(1)#Bx1xS
        output = torch.bmm(atten_weights, encoder_outputs).squeeze(1) # Bx1xH
        
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size, dropout, max_length):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.max_length = max_length
        
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attn = Attn(hidden_size)
        self.dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)
        
    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(-1, 1, self.hidden_size)
        embedded = self.dropout(embedded)
            
    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)
                

NameError: name 'nn' is not defined